# HW2- Binary Classification 
- Detravious Jamari Brinkley
- CSCI-544: Applied Natural Language Processing
- python version: 3.11.4

In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

import sklearn
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brinkley97/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Data

In [4]:
dataset = "../datasets/amazon_reviews_us_Office_Products_v1_00.tsv"
amazon_reviews_copy_df = pd.read_csv(dataset, sep='\t', on_bad_lines='skip', low_memory=False)

## Keep Reviews and Ratings

In [5]:
reviews_ratings_df = amazon_reviews_copy_df.loc[0:, ['star_rating', 'review_body']]
reviews_ratings_df.reset_index(drop=True)

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [6]:
def generate_sample_reviews(df: pd.DataFrame, review_col_name: str, number_of_reviews: int = 3):
    """Include reviews and ratings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    review_col_name: `str`
        The specific_column to get the reviews and ratings of
    
    number_of_reviews: `int`
        Number of samples to include


    Return
    ------
    Nothing; instead, print the reviews with ratings
    """


    columns_to_include = [review_col_name, 'star_rating']

    # Initialize an empty list to store dictionaries
    list_of_dicts = []

    # Iterate over the specified columns and retrieve the first three rows
    for row in df[columns_to_include].head(3).to_dict(orient='records'):
        list_of_dicts.append({'star_rating': row['star_rating'], review_col_name: row[review_col_name]})

    for dictionary in list_of_dicts:
        print(dictionary)

 ## Select 100000 reviews randomly from positive and negative classes


In [7]:
def update_data_type(df: pd.DataFrame, col_name: str):
    """Update the data type of the star ratings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with rating values

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the new sentiment appened

    """

    valid_ratings = ['1','2','3','4','5']
    star_rating_series = df[col_name].copy()

    # Convert type to strings
    star_rating_series.astype('str')

    # Check valid list and see which of our stars match
    rows = star_rating_series.index
    is_rating_in_valid_ratings = rows[star_rating_series.isin(valid_ratings)]

    # Convert to list
    is_rating_in_valid_ratings = is_rating_in_valid_ratings.to_list()

    updated_df = df.iloc[is_rating_in_valid_ratings]
    updated_df[col_name] = updated_df[col_name].astype(int)
    return updated_df

In [8]:
updated_reviews_ratings_df = update_data_type(reviews_ratings_df, 'star_rating')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_22126/2907883124.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_df[col_name] = updated_df[col_name].astype(int)


In [9]:
updated_reviews_ratings_df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [10]:
updated_reviews_ratings_df = updated_reviews_ratings_df.dropna()
updated_reviews_ratings_df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [11]:
# Check for NaN values
nan_check = updated_reviews_ratings_df.isna()

# Display the DataFrame with True where NaN values exist
print(nan_check)

# Check if any NaN value exists in the DataFrame
if nan_check.any().any():
    print("There are NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")

         star_rating  review_body
0              False        False
1              False        False
2              False        False
3              False        False
4              False        False
...              ...          ...
2640249        False        False
2640250        False        False
2640251        False        False
2640252        False        False
2640253        False        False

[2640080 rows x 2 columns]
There are no NaN values in the DataFrame.


In [12]:
print("# reviews per rating", updated_reviews_ratings_df['star_rating'].value_counts())

# reviews per rating star_rating
5    1582704
4     418348
1     306967
3     193680
2     138381
Name: count, dtype: int64


In [13]:
def sample_star_ratings(df: pd.DataFrame, col_name: str, star_value: int, number_of_reviews: int):
    """Build a subset balanced dataset with reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The dataframe to use
    col_name: `str`
        The name of the column to get reviews from
    star_value: `int`
        The star rating of the review
    number_of_reviews: `int`
        The number of sub reviews to include in sample

    Return
    ------
    rating_df, sampled_rating_df: `tuple`
        All reviews with that rating and the subset reviews with that rating
    """
    
    rating_df = df[df[col_name] == star_value]
    sampled_rating_df = rating_df.sample(n=number_of_reviews)
    return rating_df, sampled_rating_df

In [14]:
subset_reviews = 50000

one_star = 1
rating_one, rating_one_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', one_star, subset_reviews)
two_stars = 2
rating_two, rating_two_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', two_stars, subset_reviews)
three_stars = 3
rating_three, rating_three_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', three_stars, subset_reviews)
four_stars = 4
rating_four, rating_four_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', four_stars, subset_reviews)
five_stars = 5
rating_five, rating_five_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', five_stars, subset_reviews)

In [15]:
sampled_reviews_df = pd.concat([rating_one_sampled, rating_two_sampled, rating_three_sampled, rating_four_sampled, rating_five_sampled])

In [16]:
sampled_reviews_df

,star_rating,review_body
201334,1,Registration is impossible. I had to call Cano...
2478780,1,I bought this based on reading the reviews on ...
2589705,1,I bought this recorder for college. At first ...
298532,1,Does not work with my Windows 8.1 Surface Pro ...
1365546,1,Calculator was defective. Some numbers would n...
...,...,...
1429195,5,"I purchased a Purple Cows laminator, which I u..."
1206069,5,Inexpensive.. can have many and never go searc...
412021,5,A described.
2039714,5,Have used this diary for many years. Good qual...


In [17]:

def separate_reviews_by_rating(df: pd.DataFrame, rating_col: str, threshold: int, sentiment_type: str):
    """Categorizes reviews by adding a rating

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    rating_col: `str`
        Column with rating values
    
    threshold: `int`
        Where to split the ratings such that categories can be formed

    sentiment_type: `str`
        One of three types of sentiment: positive, negative, or neural

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the new sentiment appened
    """


    if sentiment_type == 'negative_review_class':
        positive_review_threshold = df[rating_col].astype('int32') > threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 1

    elif sentiment_type == 'neutral_review_class':
        positive_review_threshold = df[rating_col].astype('int32') == threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 2

    elif sentiment_type == 'positive_review_class':
        positive_review_threshold = df[rating_col].astype('int32') < threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 3
        
    return df

In [18]:
negative_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'negative_review_class')
neutral_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'neutral_review_class')
positive_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'positive_review_class')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_22126/2166075515.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 1
/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_22126/2166075515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 2
/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_22126/2166075515.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [19]:
sampled_reviews_ratings_df = pd.concat([negative_review_class_df, neutral_review_class_df, positive_review_class_df])
sampled_reviews_ratings_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN
...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0


In [20]:
negative_reviews_df = sampled_reviews_ratings_df['negative_review_class'].dropna()
neutral_reviews_df = sampled_reviews_ratings_df['neutral_review_class'].dropna()
positive_reviews_df = sampled_reviews_ratings_df['positive_review_class'].dropna()

In [21]:
sampled_reviews_ratings_df['review_class'] = pd.concat([negative_reviews_df, neutral_reviews_df, positive_reviews_df])

In [22]:
sampled_reviews_ratings_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN,1.0
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN,1.0
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN,1.0
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN,1.0
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0,3.0
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0,3.0
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0,3.0
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0,3.0


# Ignore below

# Data Cleaning

## Lower case

In [23]:
def convert_reviews_to_lower_case(df: pd.DataFrame, col_name: str):
    """Convert all reviews to lower case

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the lower cased reviews
    """
    
    lower_case_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    
    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_reviews_idx, type(text_review), text_review)

        # NOT all reviews are strings, thus all can't be converted to lower cased
        if type(text_review) != str:
            print(True, text_review)
            converted_str = str(text_review)
            lower_case_reviews.append(text_review)
         
        else:
            update_text_review = text_review.lower()
            lower_case_reviews.append(update_text_review)

    updated_df['lower_cased'] = lower_case_reviews
    return updated_df

In [24]:
reviews_lower_cased = convert_reviews_to_lower_case(sampled_reviews_ratings_df, 'review_body')

In [25]:
reviews_lower_cased

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN,1.0,good for the $. sometimes the unit gets a lit...
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN,1.0,"this is an ok replacement, but it should have ..."
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN,1.0,purchased for organizing and carrying numerous...
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN,1.0,"i like the size of numbers , how it looks but ..."
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN,1.0,didn't leave five stars only because they look...
...,...,...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0,3.0,this sharpener looks great but kept breaking t...
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0,3.0,i ordered this thinking it looked good for the...
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0,3.0,"the damned thing leaks, holds too little ink, ..."
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0,3.0,for nearly triple the price of a translucent g...


In [26]:
print("reviews_lower_cased:")
generate_sample_reviews(reviews_lower_cased, 'lower_cased', 3)

reviews_lower_cased:
{'star_rating': 4, 'lower_cased': 'good for the $.  sometimes the unit gets a little fincky when the bin is full.  the door is a little tricky to close sometimes, but overall a good shredder.'}
{'star_rating': 4, 'lower_cased': 'this is an ok replacement, but it should have the guard as the original does. other than that, it will serve the purpose for now.'}
{'star_rating': 4, 'lower_cased': 'purchased for organizing and carrying numerous picture exchange communication system cards.  great way to keep them clean and available, very good price for what you gain, small enough to carry multiple sets on long outing or trips.'}


## Remove HTML and URLs

In [27]:
def remove_html_and_urls(df: pd.DataFrame, col_name: str):
    """Remove HTML and URLs from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the html_and_urls removed
    """
    
    # url_pattern = re.compile(r'https?://\S+|www\. \S+')

    cleaned_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
            # Check and remove HTML tags
            has_html = bool(re.search('<.*?>', text_review))
            if has_html == True:
                # print("Review", text_reviews_idx, "has HTML -- ", text_review)
                pass

            no_html_review = re.sub('<.*?>', ' ', text_review)
            # print("Review", text_reviews_idx, "without HTML -- ", no_html_review)
        
            # Check and remove URLs
            has_url = bool(re.search(r'http\S+', no_html_review))
            if has_url == True:
                # print("Review", text_reviews_idx, "has URL --", no_html_review)
                pass

            no_html_url_review = re.sub(r'http\S+', '', no_html_review)
            # print("Review", text_reviews_idx, "without HTML, URL -- ", no_html_url_review)
            # print()
            cleaned_reviews.append(no_html_url_review)
        else:
            # print(text_reviews_idx, text_review)
            cleaned_reviews.append(text_review)
            

    updated_df['without_html_urls'] = cleaned_reviews
    return updated_df

In [28]:
no_html_urls_df = remove_html_and_urls(reviews_lower_cased, 'lower_cased')

In [29]:
no_html_urls_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN,1.0,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a lit...
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN,1.0,"this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ..."
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN,1.0,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN,1.0,"i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ..."
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN,1.0,didn't leave five stars only because they look...,didn't leave five stars only because they look...
...,...,...,...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0,3.0,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0,3.0,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0,3.0,"the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ..."
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0,3.0,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...


In [30]:
print("without_html_urls:")
generate_sample_reviews(no_html_urls_df, 'without_html_urls', 3)

without_html_urls:
{'star_rating': 4, 'without_html_urls': 'good for the $.  sometimes the unit gets a little fincky when the bin is full.  the door is a little tricky to close sometimes, but overall a good shredder.'}
{'star_rating': 4, 'without_html_urls': 'this is an ok replacement, but it should have the guard as the original does. other than that, it will serve the purpose for now.'}
{'star_rating': 4, 'without_html_urls': 'purchased for organizing and carrying numerous picture exchange communication system cards.  great way to keep them clean and available, very good price for what you gain, small enough to carry multiple sets on long outing or trips.'}


## Remove Contractions

In [31]:
store_contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they're": "they are",
    "wasn't": "was not",
    "we're": "we are",
    "weren't": "were not",
    "won't": "will not",
    "wouldn't": "would not",
    "you're": "you are",
    "you'll": "you will",
    "you'd": "you would",
    "we'll": "we will",
    "we've": "we have",
    "we'd": "we would",
    "I'm": "I am",
    "i've": "I have",
    "I've": "I have",
    "I'd": "I would",
    "it'll": "it will",
    "they'll": "they will",
    "they've": "they have",
    "they'd": "they would",
    "he'll": "he will",
    "he'd": "he would",
    "she'll": "she will",
    "we'd": "we would",
    "we'll": "we will",
    "you've": "you have",
    "you'd": "you would",
    "you'll": "you will",
    "I'll": "I will",
    "I'd": "I would",
    "it's": "it is",
    "it'd": "it would",
    "i'm": "I am",
    "he's": "he is",
    "he'll": "he will",
    "she's": "she is",
    "she'll": "she will",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "you're": "you are",
    "you've": "you have",
    "you'll": "you will",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "that's": "that is",
    "that'll": "that will",
    "that'd": "that would",
    "who's": "who is",
    "who'll": "who will",
    "who'd": "who would",
    "what's": "what is",
    "what'll": "what will",
    "what'd": "what would",
    "when's": "when is",
    "when'll": "when will",
    "when'd": "when would",
    "where's": "where is",
    "where'll": "where will",
    "where'd": "where would",
    "why's": "why is",
    "why'll": "why will",
    "why'd": "why would",
    "how's": "how is",
    "how'll": "how will",
    "how'd": "how would"
}


In [32]:
def locate_and_replace_contractions(review):
    """Find the contractions to replace from a specific review

    Parameters
    ----------
    review: `str`
        A specific review

    Return
    ------
    non_contraction_review: `str`
        The updated specific review with contractions expanded
    
    """
    if isinstance(review, str):
        get_words = review.split()

        store_non_contraction_words = []

        for word in get_words:
            if word in store_contractions:
                non_contraction_form = store_contractions[word]
                # print(word, "-->", non_contraction_form)

                store_non_contraction_words.append(non_contraction_form)

            else:
                # print(word)
                store_non_contraction_words.append(word)

        non_contraction_review = ' '.join(store_non_contraction_words)
        return non_contraction_review
    else:
        return review


In [33]:
def remove_contractions(df:pd.DataFrame, col_name: str):
    """Remove contractions from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    without_contractions_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        # print("Review", text_reviews_idx, "with possible contraction(s) -- ", text_review)

        without_contraction = locate_and_replace_contractions(text_review)

        # print("Review", text_reviews_idx, "without contraction -- ", without_contraction)
        # print()

        without_contractions_reviews.append(without_contraction)

    updated_df['without_contractions'] = without_contractions_reviews
    return updated_df

In [34]:
no_contractions_df = remove_contractions(no_html_urls_df, 'without_html_urls')

In [35]:
no_contractions_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN,1.0,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a litt...
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN,1.0,"this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ..."
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN,1.0,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN,1.0,"i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ..."
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN,1.0,didn't leave five stars only because they look...,didn't leave five stars only because they look...,did not leave five stars only because they loo...
...,...,...,...,...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0,3.0,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0,3.0,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0,3.0,"the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ..."
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0,3.0,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...


In [36]:
print("without_contractions:")
generate_sample_reviews(no_contractions_df, 'without_contractions', 3)

without_contractions:
{'star_rating': 4, 'without_contractions': 'good for the $. sometimes the unit gets a little fincky when the bin is full. the door is a little tricky to close sometimes, but overall a good shredder.'}
{'star_rating': 4, 'without_contractions': 'this is an ok replacement, but it should have the guard as the original does. other than that, it will serve the purpose for now.'}
{'star_rating': 4, 'without_contractions': 'purchased for organizing and carrying numerous picture exchange communication system cards. great way to keep them clean and available, very good price for what you gain, small enough to carry multiple sets on long outing or trips.'}


## Remove Non-alphabetical characters

In [37]:
def remove_non_alphabetical_characters(df:pd.DataFrame, col_name: str):
    """Remove Non-alphabetical characters from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the non-alphabetical characters removed
    """

    alphabetical_char_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        
        if isinstance(text_review, str):

            # Check for non-alphabetical characters
            has_non_alphabetical_char = bool(re.search(r'[^a-zA-Z]', text_review))
            if has_non_alphabetical_char == True:
                # print("Review", text_reviews_idx, "has HTML -- ", text_review)
                pass
            
            # Remove non-alphabetical characters
            with_alphabetical_char = re.sub(r'[^a-zA-Z\s]', ' ', text_review)
            # print("Review", text_reviews_idx, "has HTML -- ", with_alphabetical_char)
            alphabetical_char_reviews.append(with_alphabetical_char)
        else:
            alphabetical_char_reviews.append(text_review)

    updated_df['with_alpha_chars_only'] = alphabetical_char_reviews
    return updated_df

In [38]:
only_alpha_chars_df = remove_non_alphabetical_characters(no_contractions_df, 'without_contractions')

In [39]:
only_alpha_chars_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN,1.0,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a litt...,good for the sometimes the unit gets a litt...
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN,1.0,"this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...",this is an ok replacement but it should have ...
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN,1.0,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN,1.0,"i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...",i like the size of numbers how it looks but ...
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN,1.0,didn't leave five stars only because they look...,didn't leave five stars only because they look...,did not leave five stars only because they loo...,did not leave five stars only because they loo...
...,...,...,...,...,...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0,3.0,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0,3.0,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0,3.0,"the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...",the damned thing leaks holds too little ink ...
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0,3.0,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...


In [40]:
print("with_alpha_chars_only:")
generate_sample_reviews(only_alpha_chars_df, 'with_alpha_chars_only', 3)

with_alpha_chars_only:
{'star_rating': 4, 'with_alpha_chars_only': 'good for the    sometimes the unit gets a little fincky when the bin is full  the door is a little tricky to close sometimes  but overall a good shredder '}
{'star_rating': 4, 'with_alpha_chars_only': 'this is an ok replacement  but it should have the guard as the original does  other than that  it will serve the purpose for now '}
{'star_rating': 4, 'with_alpha_chars_only': 'purchased for organizing and carrying numerous picture exchange communication system cards  great way to keep them clean and available  very good price for what you gain  small enough to carry multiple sets on long outing or trips '}


## Remove extra spaces

In [41]:
def remove_extra_spaces(df:pd.DataFrame, col_name: str):
    """Remove extra spaces from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    single_spaced_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
        # Check if there are any extra spaces
            has_extra_space = bool(re.search(r' +', text_review))
            if has_extra_space == True:
                # print("Review", text_reviews_idx, "has extra space -- ", text_review)
                pass
            
            # Remove extra spaces
            single_spaced_review = re.sub(r' +', ' ', text_review)
            # print("Review", text_reviews_idx, "without extra space -- ", single_spaced_review)
            # print()
            
            single_spaced_reviews.append(single_spaced_review)
        else:
            single_spaced_reviews.append(text_review)

    updated_df['without_extra_space'] = single_spaced_reviews
    return updated_df

In [42]:
no_extra_space_df = remove_extra_spaces(only_alpha_chars_df, 'with_alpha_chars_only')

In [43]:
no_extra_space_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN,1.0,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a litt...,good for the sometimes the unit gets a litt...,good for the sometimes the unit gets a little ...
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN,1.0,"this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...",this is an ok replacement but it should have ...,this is an ok replacement but it should have t...
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN,1.0,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN,1.0,"i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...",i like the size of numbers how it looks but ...,i like the size of numbers how it looks but it...
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN,1.0,didn't leave five stars only because they look...,didn't leave five stars only because they look...,did not leave five stars only because they loo...,did not leave five stars only because they loo...,did not leave five stars only because they loo...
...,...,...,...,...,...,...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0,3.0,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0,3.0,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0,3.0,"the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...",the damned thing leaks holds too little ink ...,the damned thing leaks holds too little ink bu...
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0,3.0,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...


In [44]:
print("without_extra_space:")
generate_sample_reviews(no_extra_space_df, 'without_extra_space', 3)

without_extra_space:
{'star_rating': 4, 'without_extra_space': 'good for the sometimes the unit gets a little fincky when the bin is full the door is a little tricky to close sometimes but overall a good shredder '}
{'star_rating': 4, 'without_extra_space': 'this is an ok replacement but it should have the guard as the original does other than that it will serve the purpose for now '}
{'star_rating': 4, 'without_extra_space': 'purchased for organizing and carrying numerous picture exchange communication system cards great way to keep them clean and available very good price for what you gain small enough to carry multiple sets on long outing or trips '}


# Pre-processing

## remove the stop words 

In [45]:
def filter_stop_words(df:pd.DataFrame, col_name: str):
    """Filter stop words out from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    without_stop_words_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    stop_words = set(stopwords.words("english"))

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
            text_review_words = word_tokenize(text_review) 

        

            # print("Before stop word removal", text_reviews_idx, " -- ", text_review)

            filtered_review = []

            for text_review_words_idx in range(len(text_review_words)):
                text_review_word = text_review_words[text_review_words_idx]
                
                # Check if review word is a stop word
                if text_review_word in stop_words:
                    # print("  Stop word -- ", text_review_word)
                    pass
                else:
                    # print(text_review_word, " -- is NOT a stop word in review")
                    filtered_review.append(text_review_word)

            
            filtered_review = " ".join(filtered_review)
            # print("After stop word removal", text_reviews_idx, " -- ", filtered_review)
            # print()
            
            without_stop_words_reviews.append(filtered_review)
        else:
            without_stop_words_reviews.append(text_review)
        

    updated_df['without_stop_words'] = without_stop_words_reviews
    return updated_df

In [46]:
no_stop_words_df = filter_stop_words(no_extra_space_df, 'without_extra_space')

In [47]:
no_stop_words_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space,without_stop_words
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN,1.0,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a litt...,good for the sometimes the unit gets a litt...,good for the sometimes the unit gets a little ...,good sometimes unit gets little fincky bin ful...
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN,1.0,"this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...",this is an ok replacement but it should have ...,this is an ok replacement but it should have t...,ok replacement guard original serve purpose
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN,1.0,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased organizing carrying numerous picture...
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN,1.0,"i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...",i like the size of numbers how it looks but ...,i like the size of numbers how it looks but it...,like size numbers looks running clock noise li...
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN,1.0,didn't leave five stars only because they look...,didn't leave five stars only because they look...,did not leave five stars only because they loo...,did not leave five stars only because they loo...,did not leave five stars only because they loo...,leave five stars looked sort crappy riding bac...
...,...,...,...,...,...,...,...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0,3.0,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,sharpener looks great kept breaking tip pencil...
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0,3.0,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,ordered thinking looked good less price bit st...
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0,3.0,"the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...",the damned thing leaks holds too little ink ...,the damned thing leaks holds too little ink bu...,damned thing leaks holds little ink great nib
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0,3.0,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,nearly triple price translucent green cheap st...


In [48]:
print("without_stop_words:")
generate_sample_reviews(no_stop_words_df, 'without_stop_words', 3)

without_stop_words:
{'star_rating': 4, 'without_stop_words': 'good sometimes unit gets little fincky bin full door little tricky close sometimes overall good shredder'}
{'star_rating': 4, 'without_stop_words': 'ok replacement guard original serve purpose'}
{'star_rating': 4, 'without_stop_words': 'purchased organizing carrying numerous picture exchange communication system cards great way keep clean available good price gain small enough carry multiple sets long outing trips'}


## perform lemmatization  

- "A sentence with many words"
    - "words" -> word

In [49]:
def lemmentize_review(df:pd.DataFrame, col_name: str):
    """Lemmentize all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    lemmed_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    lem = WordNetLemmatizer()

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]   
        if isinstance(text_review, str):     
            words_in_review = word_tokenize(text_review) 

            # print("Before lem update", text_reviews_idx, " -- ", text_review)
            # print("Lemmed words", words_in_review)
            

            lemmed_sentence = []

            # Split review into words
            for lemmed_words_idx in range(len(words_in_review)):
                word = words_in_review[lemmed_words_idx]
                
                apply_lemmatization = lem.lemmatize(word)
                # print(apply_lemmatization)
                
                lemmed_sentence.append(apply_lemmatization)
                filtered_review = " ".join(lemmed_sentence)
        
            # print("After lem update -- ", filtered_review)
            # print()

            lemmed_reviews.append(filtered_review)
        else:
            lemmed_reviews.append(text_review)

    updated_df['lemmed_reviews'] = lemmed_reviews
    return updated_df

In [50]:
lemmed_df = lemmentize_review(no_stop_words_df, 'without_stop_words')

In [51]:
lemmed_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space,without_stop_words,lemmed_reviews
723766,4,Good for the $. Sometimes the unit gets a lit...,1.0,NaN,NaN,1.0,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a lit...,good for the $. sometimes the unit gets a litt...,good for the sometimes the unit gets a litt...,good for the sometimes the unit gets a little ...,good sometimes unit gets little fincky bin ful...,good sometimes unit get little fincky bin full...
1567767,4,"This is an OK replacement, but it should have ...",1.0,NaN,NaN,1.0,"this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...","this is an ok replacement, but it should have ...",this is an ok replacement but it should have ...,this is an ok replacement but it should have t...,ok replacement guard original serve purpose,ok replacement guard original serve purpose
1858814,4,Purchased for organizing and carrying NUMEROUS...,1.0,NaN,NaN,1.0,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased for organizing and carrying numerous...,purchased organizing carrying numerous picture...,purchased organizing carrying numerous picture...
860020,4,"I like the size of numbers , how it looks but ...",1.0,NaN,NaN,1.0,"i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...","i like the size of numbers , how it looks but ...",i like the size of numbers how it looks but ...,i like the size of numbers how it looks but it...,like size numbers looks running clock noise li...,like size number look running clock noise litt...
1436954,4,Didn't leave five stars only because they look...,1.0,NaN,NaN,1.0,didn't leave five stars only because they look...,didn't leave five stars only because they look...,did not leave five stars only because they loo...,did not leave five stars only because they loo...,did not leave five stars only because they loo...,leave five stars looked sort crappy riding bac...,leave five star looked sort crappy riding back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947395,2,This sharpener looks great but kept breaking t...,NaN,NaN,3.0,3.0,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,this sharpener looks great but kept breaking t...,sharpener looks great kept breaking tip pencil...,sharpener look great kept breaking tip pencil ...
1770089,2,I ordered this thinking it looked good for the...,NaN,NaN,3.0,3.0,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,i ordered this thinking it looked good for the...,ordered thinking looked good less price bit st...,ordered thinking looked good le price bit stra...
1232074,2,"The damned thing leaks, holds too little ink, ...",NaN,NaN,3.0,3.0,"the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...","the damned thing leaks, holds too little ink, ...",the damned thing leaks holds too little ink ...,the damned thing leaks holds too little ink bu...,damned thing leaks holds little ink great nib,damned thing leak hold little ink great nib
2371776,2,For nearly triple the price of a translucent g...,NaN,NaN,3.0,3.0,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a translucent g...,for nearly triple the price of a transluc

In [52]:
print("without_unlemmed_words:")
generate_sample_reviews(lemmed_df, 'lemmed_reviews', 3)

without_unlemmed_words:
{'star_rating': 4, 'lemmed_reviews': 'good sometimes unit get little fincky bin full door little tricky close sometimes overall good shredder'}
{'star_rating': 4, 'lemmed_reviews': 'ok replacement guard original serve purpose'}
{'star_rating': 4, 'lemmed_reviews': 'purchased organizing carrying numerous picture exchange communication system card great way keep clean available good price gain small enough carry multiple set long outing trip'}


In [53]:
# class CleanData:
#     """Perform lower case, remove HTML and URLs, remove contractions, remove non-alphabetical characters, remove extra spaces, remove stop words, and lemmatize"""


# Extract Word Embeddings

In [54]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import gensim.downloader as api
pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

2024-02-06 15:20:33,996 : INFO : loading projection weights from /Users/brinkley97/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2024-02-06 15:21:08,581 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /Users/brinkley97/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2024-02-06T15:21:08.581450', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.6.3-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [55]:
# retrieve the vocabulary of a model
def get_model_vocabulary(get_model):
    words_in_model = []
    for index, word in enumerate(get_model.index_to_key):
        words_in_model.append(word)
        # if index == 7:
        #     break
        # print(f"word #{index}/{len(get_model.index_to_key)} is {word}")
    return words_in_model

In [56]:
pre_trained_words = get_model_vocabulary(pretrained_word_two_vec_model)
pre_trained_words[:3]

['</s>', 'in', 'for']

## My Model

In [57]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __init__(self, df: pd.DataFrame, col_name: str):
        self.df = df
        self.col_name = col_name

    def __iter__(self):
        """

        Parameters
        ----------
        df: `pd.DataFrame`
            The data
        
        col_name: `str`
            Column with reviews

        words_in_model: `list`
            Words in Word2Vec model

        Return
        ------
        
        """

        text_reviews = self.df[self.col_name].values

        for text_reviews_idx in range(len(text_reviews)):
            text_review = text_reviews[text_reviews_idx]
            # print(text_reviews_idx, "--", text_review)

            yield utils.simple_preprocess(text_review)

        

2024-02-06 15:21:08,831 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-02-06 15:21:08,832 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2024-02-06 15:21:08,832 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2024-02-06T15:21:08.832950', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.6.3-x86_64-i386-64bit', 'event': 'created'}


In [58]:
import gensim.models

sentences = MyCorpus(lemmed_df, 'lemmed_reviews')
print("\nSentences", sentences)
my_model = gensim.models.Word2Vec(sentences=sentences, vector_size=300, window=11, min_count=10)

2024-02-06 15:21:08,915 : INFO : collecting all words and their counts
2024-02-06 15:21:08,916 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types



Sentences <__main__.MyCorpus object at 0x200b17ed0>


2024-02-06 15:21:09,292 : INFO : PROGRESS: at sentence #10000, processed 313585 words, keeping 14356 word types
2024-02-06 15:21:09,643 : INFO : PROGRESS: at sentence #20000, processed 623535 words, keeping 19573 word types
2024-02-06 15:21:10,000 : INFO : PROGRESS: at sentence #30000, processed 939223 words, keeping 23531 word types
2024-02-06 15:21:10,364 : INFO : PROGRESS: at sentence #40000, processed 1252125 words, keeping 26823 word types
2024-02-06 15:21:10,729 : INFO : PROGRESS: at sentence #50000, processed 1569345 words, keeping 29687 word types
2024-02-06 15:21:11,007 : INFO : PROGRESS: at sentence #60000, processed 1784455 words, keeping 31748 word types
2024-02-06 15:21:11,279 : INFO : PROGRESS: at sentence #70000, processed 1997856 words, keeping 33702 word types
2024-02-06 15:21:11,619 : INFO : PROGRESS: at sentence #80000, processed 2207313 words, keeping 35505 word types
2024-02-06 15:21:11,875 : INFO : PROGRESS: at sentence #90000, processed 2424880 words, keeping 372

### Similar scores

[ ] Write summary of differences between their model and my model

### PRetrained

In [59]:
# pretrained_word_two_vec_model['hello'].shape

result = pretrained_word_two_vec_model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
result


[('queen', 0.7118193507194519)]

### Our model

In [60]:
# Must extract our word vectors from our model 
# my_model.wv['hello']


my_result = my_model.wv.most_similar(positive=['woman', 'king'], negative=['man', 'men'], topn=10)
my_result

[('chisel', 0.41360870003700256),
 ('reduced', 0.3794267773628235),
 ('magnum', 0.37896981835365295),
 ('competitive', 0.35803303122520447),
 ('varies', 0.3496881127357483),
 ('quartet', 0.34956613183021545),
 ('similar', 0.3480735421180725),
 ('pk', 0.34597742557525635),
 ('sub', 0.34452173113822937),
 ('based', 0.3424023687839508)]

In [61]:
my_model_words = get_model_vocabulary(my_model.wv)
my_model_words[:3]

['printer', 'one', 'work']

In [62]:
# def extract_word_embeddings(sentence: str, words_in_model, get_model):
#     """Extract word embeddings per sentence
    
#     """
#     sentence_vectorized = []

#     # print("Before -- ", sentence, "\n")
#     # print(len(updated_sentence))

#     # Split review into words
#     for word in sentence.split(" "):
#         # print(word)

#         if word in words_in_model:
#             # print(word)
#             vector = get_model[word]
#             # print(vector)
#             sentence_vectorized.append(vector)
#         # else:
#             # print(False, word)
#     return sentence_vectorized


In [197]:
def word_embeddings(df: pd.DataFrame, col_name: str, words_in_model: list, model_to_use):
    """Extract word embeddings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    words_in_model: `list`
        Words in Word2Vec model

    Return
    ------
    
    """
    
    sentence_vectorized = []
    mean_sentences_vectorized = []
    sentences = df[col_name].values

    for sentences_idx in range(len(sentences)):
        vectorized_words = []
        sentence = sentences[sentences_idx]
        # print("Sentence", sentences_idx, "Pre-vectorized -- ", sentence)
        for word in sentence.split(" "):
            if word in words_in_model:
                vector_of_word = model_to_use[word]
                vectorized_words.append(vector_of_word)
                # print("--->", word, "is in model with vector lenght of", len(vector_of_word))
                # words_vectorized.append(vector_of_word)
        sentence_vectorized.append(vectorized_words)
        # print("Sentence", sentences_idx, "Post-vectorized \n")

        mean_of_sentence = np.mean(sentence_vectorized[sentences_idx], axis=0)
        # print(len(mean_of_sentence), mean_of_sentence)
        mean_sentences_vectorized.append(mean_of_sentence)


    return mean_sentences_vectorized

In [198]:
# len(pre_trained_words), len(my_model_words)

In [200]:
# print("Pretrained")
pretrained_embeddings = word_embeddings(lemmed_df[:3], 'lemmed_reviews', pre_trained_words, pretrained_word_two_vec_model)

# print("None-pretrained")
my_embeddings = word_embeddings(lemmed_df[:3], 'lemmed_reviews', my_model_words, my_model.wv)

In [213]:
embeddings_df = lemmed_df.iloc[0:3, [5, 1, 12]]
embeddings_df['pretrained_embeddings'] = pretrained_embeddings
embeddings_df['my_embeddings'] = my_embeddings
embeddings_df

,review_class,review_body,lemmed_reviews,pretrained_embeddings,my_embeddings
723766,1.0,Good for the $. Sometimes the unit gets a lit...,good sometimes unit get little fincky bin full...,"[0.0932841, 0.07194379, -0.06315104, 0.1184244...","[-0.7421835, -0.5304476, 0.18416253, -0.042262..."
1567767,1.0,"This is an OK replacement, but it should have ...",ok replacement guard original serve purpose,"[0.021105448, -0.008463542, 0.12824504, 0.0178...","[-0.26373112, -0.022724157, -0.17758863, -0.34..."
1858814,1.0,Purchased for organizing and carrying NUMEROUS...,purchased organizing carrying numerous picture...,"[0.025377808, 0.055075683, -0.0674353, 0.06252...","[-0.1262355, 0.29282907, 0.17830867, 0.1272906..."


In [203]:
# length of each sentence
len(list(embeddings_df['pretrained_embeddings'])[2]), len(list(embeddings_df['my_embeddings'])[0])

(300, 300)

## Simple models

In [ ]:
sentiments = lemmed_df['sentiment']
sentiments.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf_features, sentiments, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Models + Evaluation Metrics

In [ ]:
def eval_accuracy(y_true, y_prediction):
    return sklearn.metrics.accuracy_score(y_true, y_prediction)

def eval_precision(y_true, y_prediction):
    return sklearn.metrics.precision_score(y_true, y_prediction)

def eval_recall(y_true, y_prediction):
    return sklearn.metrics.recall_score(y_true, y_prediction)

def eval_f1_score(y_true, y_prediction):
    return sklearn.metrics.f1_score(y_true, y_prediction)

In [ ]:
def train_eval_metric(y_train_true, y_train_predictions):
    accuracy = eval_accuracy(y_train_true, y_train_predictions)
    precision = eval_precision(y_train_true, y_train_predictions)
    recall = eval_recall(y_train_true, y_train_predictions)
    f1 = eval_f1_score(y_train_true, y_train_predictions)

    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return metrics_dict

def test_eval_metric(y_test_true, y_test_predictions):
    accuracy = eval_accuracy(y_test_true, y_test_predictions)
    precision = eval_precision(y_test_true, y_test_predictions)
    recall = eval_recall(y_test_true, y_test_predictions)
    f1 = eval_f1_score(y_test_true, y_test_predictions)

    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return metrics_dict

# Perceptron

In [ ]:
def perceptron_model(X_train, X_test, y_train, y_test): 

    technique = Perceptron(tol=1e-3, random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [ ]:
perceptron_train_metrics, perceptron_test_metrics = perceptron_model(X_train, X_test, y_train, y_test)

In [ ]:
perceptron_train_metrics, perceptron_test_metrics

# SVM

In [ ]:
def svm_model(X_train, X_test, y_train, y_test): 

    technique = LinearSVC(tol=1e-3, random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [ ]:
svm_train_metrics, svm_test_metrics = svm_model(X_train, X_test, y_train, y_test)

In [ ]:
svm_train_metrics, svm_test_metrics

# Logistic Regression

In [ ]:
def logistic_regression_model(X_train, X_test, y_train, y_test): 

    technique = LogisticRegression(random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics

In [ ]:
logistic_regression_train_metrics, logistic_regression_test_metrics = logistic_regression_model(X_train, X_test, y_train, y_test)

In [ ]:
logistic_regression_train_metrics, logistic_regression_test_metrics

# Naive Bayes

In [ ]:
def naive_bayes_model(X_train, X_test, y_train, y_test): 

    technique = MultinomialNB()
    technique.fit(X_train.toarray(), y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)

    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [ ]:
naive_bayes_train_metrics, naive_bayes_test_metrics = naive_bayes_model(X_train, X_test, y_train, y_test)

In [ ]:
naive_bayes_train_metrics, naive_bayes_test_metrics